
#### Packages einlesen

In [1]:
import netCDF4 as nc 
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import xarray as xr
import PIL 
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from matplotlib.colors import LinearSegmentedColormap
import pandas as pd
from scipy.stats import pearsonr



/sw/spack-levante/mambaforge-22.9.0-2-Linux-x86_64-kptncg/lib/python3.10/site-packages/pyproj/__init__.py:91: UserWarning: Valid PROJ data directory not found. Either set the path using the environmental variable PROJ_LIB or with `pyproj.datadir.set_data_dir`.
  warnings.warn(str(err))
ERROR 1: PROJ: proj_create_from_database: Open of /sw/spack-levante/mambaforge-22.9.0-2-Linux-x86_64-kptncg/share/proj failed
PROJ: proj_create_from_database: Open of /sw/spack-levante/mambaforge-22.9.0-2-Linux-x86_64-kptncg/share/proj failed


### Filepath festlegen

In [2]:
##### Filepath Observation #####
filepath_observation_files = '/work/ch0636/g300123/Calibration_data/observation_ncfiles'
selected_observation = 'GRDC-Daily'
# selected_observation = 'GRDC-Daily-Spain'

##### Filepath Simulation #####
filepath_simulation_results = '/work/ch0636/g300123/Outputs_5arcmin'
# selected_simulation = '24_02_24_landcover_5arcmin_0000_zeros'
# selected_simulation = '24_03_20_init_INPTAG_2003_LAT_0000_zeros'
selected_simulation = '24_03_23_init_INPTAG_1999to2010_LAT_0000_zeros_forvalidation'



### Festlegen, welche Jahre untersucht werden sollen 

In [3]:
# start_date = '2003-01-01' 
# end_date = '2003-12-31' 
# yearsname = '2003'

start_date = '1999-01-01' 
end_date = '2008-12-31' 
yearsname = '1999to2008'

### Festlegen, für welche Stationen es Abweichungen bei der Wahl des passenden Rasters im Vergleich zu automatischen Auswahl gibt 

In [4]:
# Manuell hier die Stations_IDs eingeben für die Veränderungen forgenommen werden müssen 
list_of_station_IDs_lat_minus1 = [ 6221102 , 6139770 , 6233650 , 6226300 , 6781800 , 6226310 , 6227130 , 6227410 ]
list_of_station_IDs_lon_minus1 = [ 6212740 ]
list_of_station_IDs_lat_plus1 = []
list_of_station_IDs_lon_plus1 = [ 6216610 ]

# Chooz (Belgium) / 6221102 lat -1 
# Neublans-Abergement (France) / 6139770 lat -1
# Sollefta (Sweden) / 6233650 lat -1 
# Castejon (Spain) / 6226300 lat -1 
# Tighina (Moldavia) / 6781800 lat -1 
#### # Alba Iulia (Rumania) / 6744201 lon +1

# 6212740  /  VALLADOLID-PISUERGA (ES) :  lon -1 
# 6216610  /  VILLANUEVA DE LA SERENA (ES) :   lon +1 
# s.o. 6226300  /  CASTEJON (ES) :   lat -1 
# 6226310  /  CAPARROSO (ES) :   lat -1
# 6227130  /  LA PRESA (ES) :   lat -1
# 6227410  /  ROJALES (ES) :   lat -1

## Oberservation nc-Datensätze

### Oberservation nc-Datensätze von GRDC einlesen

In [5]:
# nc-file in xarray einlesen
observation_file = filepath_observation_files + '/' + selected_observation + '.nc'
xr_obs = xr.open_dataset(observation_file)
xr_obs

# Liste aller einzigartigen IDs
list_of_station_IDs = xr_obs['id'].values

# Leeres Dictionary erstellen 
Dict_obs_data = {}

# Schleife über jede einzigartige ID, um die Daten für "discharge" zu isolieren
for ID in list_of_station_IDs:
    obs_data = xr_obs['runoff_mean'].sel(id = ID, time = slice (start_date, end_date)).values
    # Speichere die Daten im Dictionary unter dem entsprechenden ID-Namen
    Dict_obs_data[ID] = obs_data

    

/sw/spack-levante/mambaforge-22.9.0-2-Linux-x86_64-kptncg/lib/python3.10/site-packages/xarray/backends/plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


## Simulation nc-Datensätze

### Simulation nc-Datensätze von CWatM einlesen Teil 1/2

In [6]:
# nc-file in xarray einlesen
simulation_file = filepath_simulation_results  + '/' + selected_simulation  + '/' + 'discharge_daily.nc'
xr_sim = xr.open_dataset(simulation_file)
xr_sim 

# Liste aller einzigartigen IDs
list_of_station_IDs = xr_obs['id'].values


### lat_station_index und lon_station_index automatisch herausfinden 

In [7]:
# Geographische Koordinaten automatisch auslesen 
lon_station = {}
lat_station = {}
height_station = {}

for ID in list_of_station_IDs: 
    lon_station [ID] = xr_obs['geo_x'].sel(id = ID).values
    lat_station[ID] = xr_obs['geo_y'].sel(id = ID).values
    height_station [ID] =  xr_obs['geo_z'].sel(id = ID).values
    
# Alle Werte der latitudes und longitudes auslesen 
latitudes = xr_sim['lat'].values
longitudes = xr_sim['lon'].values

# Nächstgelegende Lat und lon index automatisch auslesen
lat_station_index = {}
lon_station_index = {}

for ID in list_of_station_IDs: 
    lat_station_index [ID] = np.abs(latitudes - lat_station[ID]).argmin()
    lon_station_index [ID] = np.abs(longitudes - lon_station[ID]).argmin()


### Dictionary für lat_station_index_fit und lon_station_index_fit erstellen

In [8]:
# Jetzt Dictionarys für lat_index_fit und lon_index_fit damit erstellen 
lat_station_index_fit = {}
lon_station_index_fit = {}

for ID in list_of_station_IDs:
    lat_station_index_fit[ID] = lat_station_index[ID]
    lon_station_index_fit[ID] = lon_station_index[ID]
    
for ID in list_of_station_IDs_lat_minus1:
    if ID in list_of_station_IDs:
        lat_station_index_fit[ID] = lat_station_index[ID] - 1 
for ID in list_of_station_IDs_lon_minus1:
    if ID in list_of_station_IDs:
        lon_station_index_fit[ID] = lon_station_index[ID] - 1
for ID in list_of_station_IDs_lat_plus1:
    if ID in list_of_station_IDs:
        lat_station_index_fit[ID] = lat_station_index[ID] + 1 
for ID in list_of_station_IDs_lon_plus1:
    if ID in list_of_station_IDs:
        lon_station_index_fit[ID] = lon_station_index[ID] + 1

# Finale Werte für lat_station_index_fit und lon_station_index_fit ausgeben
# print(lat_station_index_fit)
# print(lon_station_index_fit)


### Simulation nc-Datensätze von CWatM einlesen Teil 2/2

In [9]:
# Leeres Dictionary erstellen 
Dict_sim_data = {}

# Schleife über jede einzigartige ID, um die Daten für "discharge" zu isolieren
for ID in list_of_station_IDs:
    sim_data_everywhere = xr_sim['discharge'].sel(time = slice (start_date, end_date))
    sim_data = sim_data_everywhere[:, lat_station_index_fit[ID], lon_station_index_fit[ID]].values
    # Speichere die Daten im Dictionary unter dem entsprechenden ID-Namen
    Dict_sim_data[ID] = sim_data




## Amount of average discharge berechnen 

In [10]:
def calculate_averagedischarge_obs(stationID):
    ## netcdf-files von eigener CWatM-Simulation einlesen   
    obs_data = Dict_obs_data[stationID]
    # Berechnung der einzelnen Komponenten
    obs_mean = np.nanmean(obs_data)
    # Berechnung der AvergeDischarge
    AvergeDischarge_obs = obs_mean
    return stationID, AvergeDischarge_obs



#### Dictionary for stations

In [11]:
Dict_stationname = {}
for ID in list_of_station_IDs:
    stationname = xr_obs['station_name'].sel(id = ID).values
    country = xr_obs['country'].sel(id = ID).values
    Dict_stationname[ID] = f"{stationname} ({country})"



#### Amount of average discharge für die verschiedenen Standorte berechnen 

In [12]:
AvergeDischarge = {}
for ID in list_of_station_IDs:
    stationID,  stationAD = calculate_averagedischarge_obs(ID) 
    AvergeDischarge [ID] = np.round(stationAD, 4)
    print('Station: ' ,  ID  , ' / ' ,  Dict_stationname[ID]  , ' , ' ,  'Average discharge:' , np.round(AvergeDischarge[ID],1))



Station:  6212410  /  TORO (ES)  ,  Average discharge: 102.9
Station:  6212440  /  CARRASCAL (ES)  ,  Average discharge: 94.5
Station:  6212460  /  HERRERA DE DUERO (ES)  ,  Average discharge: 25.0
Station:  6212470  /  GORMAZ (ES)  ,  Average discharge: 12.1
Station:  6212510  /  BRETO (ES)  ,  Average discharge: 122.2
Station:  6212710  /  QUINTANA DEL PUENTE (ES)  ,  Average discharge: 24.5
Station:  6212740  /  VALLADOLID-PISUERGA (ES)  ,  Average discharge: 64.0
Station:  6212850  /  VALDESTILLAS (ES)  ,  Average discharge: 8.1
Station:  6213700  /  TALAVERA (ES)  ,  Average discharge: nan
Station:  6213750  /  ARANJUEZ (PUENTE LARGO) (ES)  ,  Average discharge: 26.4
Station:  6213780  /  MEJORADA DEL CAMPO (ES)  ,  Average discharge: 17.0
Station:  6213850  /  EMBOCADOR (ES)  ,  Average discharge: 7.6
Station:  6216510  /  AZUD DE BADAJOZ (ES)  ,  Average discharge: 38.1
Station:  6216520  /  VILLARRUBIA (ES)  ,  Average discharge: 0.4
Station:  6216610  /  VILLANUEVA DE LA SEREN

/tmp/ipykernel_1111688/2219232428.py:5: RuntimeWarning: Mean of empty slice
  obs_mean = np.nanmean(obs_data)
